In [2]:
import numpy as np
from sklearn import datasets
from sklearn import tree

# Load iris
iris = datasets.load_iris()
X = iris.data
y = iris.target

# Build decision tree classifier
dt = tree.DecisionTreeClassifier(criterion='entropy')
dt.fit(X, y)

DecisionTreeClassifier(criterion='entropy')

In [3]:
dotfile = open("dt.dot", 'w')
tree.export_graphviz(dt, out_file=dotfile, feature_names=iris.feature_names)
dotfile.close()

In [4]:
from sklearn.tree import _tree

def tree_to_code(tree, feature_names):
	"""
	Outputs a decision tree model as a Python function
	
	Parameters:
	-----------
	tree: decision tree model
		The decision tree to represent as a function
	feature_names: list
		The feature names of the dataset used for building the decision tree
	"""

	tree_ = tree.tree_
	feature_name = [
		feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
		for i in tree_.feature
	]
	print("def tree({}):".format(", ".join(feature_names)))

	def recurse(node, depth):
		indent = "  " * depth
		if tree_.feature[node] != _tree.TREE_UNDEFINED:
			name = feature_name[node]
			threshold = tree_.threshold[node]
			print("{}if {} <= {}:".format(indent, name, threshold))
			recurse(tree_.children_left[node], depth + 1)
			print("{}else:  # if {} > {}".format(indent, name, threshold))
			recurse(tree_.children_right[node], depth + 1)
		else:
			print("{}return {}".format(indent, tree_.value[node]))

	recurse(0, 1)

In [5]:
tree_to_code(dt, list(iris.feature_names))

def tree(sepal length (cm), sepal width (cm), petal length (cm), petal width (cm)):
  if petal width (cm) <= 0.800000011920929:
    return [[50.  0.  0.]]
  else:  # if petal width (cm) > 0.800000011920929
    if petal width (cm) <= 1.75:
      if petal length (cm) <= 4.950000047683716:
        if petal width (cm) <= 1.6500000357627869:
          return [[ 0. 47.  0.]]
        else:  # if petal width (cm) > 1.6500000357627869
          return [[0. 0. 1.]]
      else:  # if petal length (cm) > 4.950000047683716
        if petal width (cm) <= 1.550000011920929:
          return [[0. 0. 3.]]
        else:  # if petal width (cm) > 1.550000011920929
          if sepal length (cm) <= 6.949999809265137:
            return [[0. 2. 0.]]
          else:  # if sepal length (cm) > 6.949999809265137
            return [[0. 0. 1.]]
    else:  # if petal width (cm) > 1.75
      if petal length (cm) <= 4.8500001430511475:
        if sepal length (cm) <= 5.950000047683716:
          return [[0. 1. 0.]]

In [6]:
def tree_to_pseudo(tree, feature_names):
	"""
	Outputs a decision tree model as if/then pseudocode
	
	Parameters:
	-----------
	tree: decision tree model
		The decision tree to represent as pseudocode
	feature_names: list
		The feature names of the dataset used for building the decision tree
	"""

	left = tree.tree_.children_left
	right = tree.tree_.children_right
	threshold = tree.tree_.threshold
	features = [feature_names[i] for i in tree.tree_.feature]
	value = tree.tree_.value

	def recurse(left, right, threshold, features, node, depth=0):
		indent = "  " * depth
		if (threshold[node] != -2):
			print(indent,"if ( " + features[node] + " <= " + str(threshold[node]) + " ) {")
			if left[node] != -1:
				recurse (left, right, threshold, features, left[node], depth+1)
				print(indent,"} else {")
				if right[node] != -1:
					recurse (left, right, threshold, features, right[node], depth+1)
				print(indent,"}")
		else:
			print(indent,"return " + str(value[node]))

	recurse(left, right, threshold, features, 0)

In [7]:
tree_to_pseudo(dt, list(iris.feature_names))

 if ( petal width (cm) <= 0.800000011920929 ) {
   return [[50.  0.  0.]]
 } else {
   if ( petal width (cm) <= 1.75 ) {
     if ( petal length (cm) <= 4.950000047683716 ) {
       if ( petal width (cm) <= 1.6500000357627869 ) {
         return [[ 0. 47.  0.]]
       } else {
         return [[0. 0. 1.]]
       }
     } else {
       if ( petal width (cm) <= 1.550000011920929 ) {
         return [[0. 0. 3.]]
       } else {
         if ( sepal length (cm) <= 6.949999809265137 ) {
           return [[0. 2. 0.]]
         } else {
           return [[0. 0. 1.]]
         }
       }
     }
   } else {
     if ( petal length (cm) <= 4.8500001430511475 ) {
       if ( sepal length (cm) <= 5.950000047683716 ) {
         return [[0. 1. 0.]]
       } else {
         return [[0. 0. 2.]]
       }
     } else {
       return [[ 0.  0. 43.]]
     }
   }
 }
